## setup data path final norm

In [2]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

## Decide length of input

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [3]:
cal_length(train_path,'ZVa9c',train_label_df)

AVG Length:  118.50598372426998  file count:  4178  max_len:  2374


## load the data

In [3]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [5]:
data_array_train, data_labels_train = TraverseFiles(train_path,'ZVa9c',train_label_df,120)
data_array_test, data_labels_test = TraverseFiles(test_path,'ZVa9c',test_label_df,120)

In [6]:
x_train=np.array(data_array_train)
labels_train=np.array(data_labels_train,dtype=int)

In [7]:
x_test=np.array(data_array_test)
labels_test=np.array(data_labels_test,dtype=int)

In [8]:
print(x_train.shape)
print(labels_train.shape)
print(x_test.shape)
print(labels_test.shape)

(4178, 120, 11)
(4178,)
(1045, 120, 11)
(1045,)


## build and train model

In [9]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
lstm.fit(x_train,
    labels_train,
    epochs = 500,
    batch_size=200
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
4178/4178 [==============================] - 4s 855us/step - loss: 0.7700 - accuracy: 0.5034
Epoch 2/500
4178/4178 [==============================] - 2s 565us/step - loss: 0.7160 - accuracy: 0.5273
Epoch 3/500
4178/4178 [==============================] - 2s 565us/step - loss: 0.6989 - accuracy: 0.5419
Epoch 4/500
4178/4178 [==============================] - 2s 569us/step - loss: 0.6949 - accuracy: 0.5464
Epoch 5/500
4178/4178 [==============================] - 2s 564us/step - loss: 0.6857 - accuracy: 0.5716
Epoch 6/500
4178/4178 [==============================] - 2s 578us/step - loss: 0.6919 - accuracy: 0.5529
Epoch 7/500
4178/4178 [==============================] - 2s 578us/step - loss: 0.6850 - accuracy: 0.5704
Epoch 8/500
4178/4178 [==============================] - 2s 578us/step - loss: 0.6857 - accuracy: 0.5716
Epoch 9/500
4178/4178 [==============================] - 2s 584us/step - loss: 0.6840 - accuracy: 0.5735
Epoch 1

4178/4178 [==============================] - 3s 623us/step - loss: 0.6684 - accuracy: 0.5945
Epoch 77/500
4178/4178 [==============================] - 3s 623us/step - loss: 0.6719 - accuracy: 0.5953
Epoch 78/500
4178/4178 [==============================] - 3s 623us/step - loss: 0.6725 - accuracy: 0.5845
Epoch 79/500
4178/4178 [==============================] - 3s 621us/step - loss: 0.6739 - accuracy: 0.5871
Epoch 80/500
4178/4178 [==============================] - 3s 625us/step - loss: 0.6723 - accuracy: 0.5931
Epoch 81/500
4178/4178 [==============================] - 3s 622us/step - loss: 0.6728 - accuracy: 0.5917
Epoch 82/500
4178/4178 [==============================] - 3s 639us/step - loss: 0.6745 - accuracy: 0.5926
Epoch 83/500
4178/4178 [==============================] - 3s 635us/step - loss: 0.6711 - accuracy: 0.5871
Epoch 84/500
4178/4178 [==============================] - 3s 632us/step - loss: 0.6716 - accuracy: 0.5905
Epoch 85/500
4178/4178 [==============================] - 3

4178/4178 [==============================] - 3s 661us/step - loss: 0.6645 - accuracy: 0.6029
Epoch 229/500
4178/4178 [==============================] - 3s 658us/step - loss: 0.6662 - accuracy: 0.5984
Epoch 230/500
4178/4178 [==============================] - 3s 664us/step - loss: 0.6665 - accuracy: 0.5950
Epoch 231/500
4178/4178 [==============================] - 3s 658us/step - loss: 0.6674 - accuracy: 0.5989
Epoch 232/500
4178/4178 [==============================] - 3s 661us/step - loss: 0.6654 - accuracy: 0.6010
Epoch 233/500
4178/4178 [==============================] - 3s 661us/step - loss: 0.6645 - accuracy: 0.6051
Epoch 234/500
4178/4178 [==============================] - 3s 657us/step - loss: 0.6634 - accuracy: 0.6036
Epoch 235/500
4178/4178 [==============================] - 3s 657us/step - loss: 0.6631 - accuracy: 0.6115
Epoch 236/500
4178/4178 [==============================] - 3s 661us/step - loss: 0.6654 - accuracy: 0.6010
Epoch 237/500
4178/4178 [==========================

4178/4178 [==============================] - 2s 573us/step - loss: 0.6624 - accuracy: 0.6120
Epoch 381/500
4178/4178 [==============================] - 2s 570us/step - loss: 0.6606 - accuracy: 0.6058
Epoch 382/500
4178/4178 [==============================] - 2s 569us/step - loss: 0.6574 - accuracy: 0.6082
Epoch 383/500
4178/4178 [==============================] - 2s 569us/step - loss: 0.6566 - accuracy: 0.6103
Epoch 384/500
4178/4178 [==============================] - 2s 569us/step - loss: 0.6599 - accuracy: 0.6127
Epoch 385/500
4178/4178 [==============================] - 2s 570us/step - loss: 0.6589 - accuracy: 0.6065
Epoch 386/500
4178/4178 [==============================] - 2s 579us/step - loss: 0.6581 - accuracy: 0.6166
Epoch 387/500
4178/4178 [==============================] - 2s 566us/step - loss: 0.6582 - accuracy: 0.6120
Epoch 388/500
4178/4178 [==============================] - 2s 568us/step - loss: 0.6593 - accuracy: 0.6103
Epoch 389/500
4178/4178 [==========================

In [14]:
lstm.save('lstm_model_ZVa9c.h5')

## Acc on the splited test

In [15]:
pred_=lstm.predict_classes(x_test)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(labels_test, pred_,digits=4))
print(accuracy_score(labels_test, pred_))

              precision    recall  f1-score   support

           0     0.5987    0.9246    0.7268       597
           1     0.6341    0.1741    0.2732       448

   micro avg     0.6029    0.6029    0.6029      1045
   macro avg     0.6164    0.5494    0.5000      1045
weighted avg     0.6139    0.6029    0.5323      1045

0.6028708133971292


## acc on cleaned test data

In [4]:
train_path = '../../../data/train'
test_path = '../../../data/test'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(cleaned_train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(cleaned_test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

In [6]:
cleaned_array_test, cleaned_labels_test = TraverseFiles(test_path,'ZVa9c',test_label_df,120)

In [7]:
model = keras.models.load_model('lstm_model_ZVa9c.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
cleaned_x=np.array(cleaned_array_test)
cleaned_labels=np.array(cleaned_labels_test,dtype=int)

In [9]:
pred_cleaned=model.predict_classes(cleaned_x)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(cleaned_labels, pred_cleaned,digits=4))
print(accuracy_score(cleaned_labels, pred_cleaned))

              precision    recall  f1-score   support

           0     0.5545    0.9508    0.7005      1668
           1     0.5119    0.0632    0.1126      1360

   micro avg     0.5522    0.5522    0.5522      3028
   macro avg     0.5332    0.5070    0.4065      3028
weighted avg     0.5354    0.5522    0.4365      3028

0.5521796565389696
